<a href="https://colab.research.google.com/github/jeffeuxMartin/CodingTipsForShare/blob/main/Colab_Advanced_Tips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratory 使用小技巧

## 1. GPU 選擇問題 

### 🤔 Issue
由於像 ML HW5 這種可能需要 train 比較久的作業，對於 GPU 本身的要求可能比較嚴苛。 

如果直接使用 K80 可能會跑非常久⋯⋯

### 😅 Original Solution
使用 `nvidia-smi` 確認 GPU 的名稱，人眼確認  
如果拿到 K80 即恢復原廠設定。

```python
! nvidia-smi
```

### 😄 Automatic Solution

因為 PyTorch 或 TensorFlow 本身需要使用 GPU 一定會知道 GPU device 名稱。  
利用這個特性可以寫判斷的程式，如果拿到不希望的 GPU 自動跳出重置，避免浪費時間用不好的 GPU 訓練。

In [ ]:
import torch
try:
    # Get GPU name, check if it's K80
    GPU_name = torch.cuda.get_device_name()
    if GPU_name[-3:] == "K80":
        print("Get K80! :'( RESTART!")
        exit()  # Restart the session
    else:
        print("Your GPU is {}!".format(GPU_name))
        print("Great! Keep going~")
except RuntimeError as e:
    if e.args == ("No CUDA GPUs are available",):
        print("You are training with CPU! "
              "Please restart!")
        exit()  # Restart the session
    else:
        print("What's wrong here?")
        print("Error message: \n", e)

Your GPU is Tesla T4!
Great! Keep going~


### 🔍 Postscript
如果因為拿到 K80 或者沒有使用 GPU 被 restart 的話，  
由於重新啟動<font color="red">不一定</font>會重新抽到其他 GPU，  
請大家從這邊<font color="blue">恢復原廠設定</font>來確保重新抽選。

Step 1. 從上方工具列選「執行階段」  
&emsp;&emsp;&emsp;![image-20210413162203255](https://tva1.sinaimg.cn/large/008eGmZEgy1gpi6yr9oi6j30ei02wwf7.jpg)

***

Step 2. 點這邊「<font color="blue">恢復原廠設定的執行階段</font>」  
&emsp;&emsp;&emsp;![image-20210413162631553](https://tva1.sinaimg.cn/large/008eGmZEgy1gpi73cmbz3j309u0cl404.jpg)


### 🚧 **Issues**
+ [ ] 無法將「恢復原廠設定」自動化  
（`google.colab` module 跟 JavaScript 操作網頁都嘗試過了）
> 如果有誰知道怎麼辦的，歡迎上 NTU COOL 分享～

## 2. Colab 斷線問題
### 🤔 Issue
由於 Colab 會偵測使用者是否在電腦前面使用，因此一定時間沒有動作會自動斷線

### 😅 Original Solution
開啟 F12 控制臺，貼上 Javascript  

![image-20210413163238760](https://tva1.sinaimg.cn/large/008eGmZEgy1gpi79q5jkgj30hy0jugp7.jpg)

```javascript
function ClickConnect() {
    console.log("Working...");  // debug 用
    // 去選擇按鈕
    var connectbutton = document.querySelector(
        "colab-toolbar-button" +
        "#connect-icon.big-icon.icon-okay");
    // 點個兩下
    connectbutton.click(); connectbutton.click();
};

// 設定固定每 10 分鐘點一下
const stopit = setInterval(
    ClickConnect, 10 * 60 * 1000);

// 這行沒有沒關係，只是避免停不下來
const stopping = stopper => clearInterval(stopper);
```

使得瀏覽器每 10 分鐘會自己按兩下這個按鈕  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<img src="https://i.imgur.com/Kuc4Lsz.png" />


### 😄 Another Solution

由於實際上網址列也可以執行 JavaScript，  
就是如果把
```javascript
console.log("Hi");
```
前面加上 `javascript:` 放在網址列變成
```
javascript:console.log("Hi");
```
就可以等效讓瀏覽器執行

只是為了安全性，通常瀏覽器會自動把 `javascript:` 截掉
***
於是，如果把一個 `j` 切掉變成 `avascript:...`   
瀏覽器就不會發現了～  
下面的 code 會產生上面那份 code 的網址列版本，只要按下  
![](https://i.imgur.com/1BlLdBk.png)  
尾巴的「<img src="https://tva1.sinaimg.cn/large/008eGmZEgy1gpi7ji4qnrj301h01mjrb.jpg" width="15"/>」按鈕
前面補上 `j` 就不必開啟 F12 控制臺了！

In [ ]:
import IPython
from google.colab import output
from IPython.display import Markdown

# display JavaScript code
display(IPython.display.Javascript('''
  var string_to_paste = (
      "avascript:function ClickConnect() {    "
    + "console.log('Working...');     "
    + "var connectbutton = document.querySelector("
    + "        'colab-toolbar-button' +       "
    + " '#connect-icon.big-icon.icon-okay');  "
    + "  connectbutton.click();   "
    + " connectbutton.click();};"
    + "const stopit = setInterval(   "
    + " ClickConnect, 10 * 60 * 1000);"
    + "const stopper = stopper => "
    + "clearInterval(stopper);")
  window.someValue = new Promise(resolve => {
    setTimeout(() => {
      resolve(string_to_paste);
    }, 100);
  });
'''))

# remind to prepend `j`
print("\x1b[01;31m請記得要在開頭加上 \"j\" ！\x1b[0m\n")
value = output.eval_js('someValue');

# display
value

<IPython.core.display.Javascript object>

請記得要在開頭加上 "j" ！



"avascript:function ClickConnect() {    console.log('Working...');     var connectbutton = document.querySelector(        'colab-toolbar-button' +        '#connect-icon.big-icon.icon-okay');    connectbutton.click();    connectbutton.click();};const stopit = setInterval(    ClickConnect, 10 * 60 * 1000);const stopper = stopper => clearInterval(stopper);"

### 🔍 Postscript

其他江湖上流傳的版本，如  
&emsp;&emsp;&emsp;https://harry0731.github.io/machine%20learning/colabdisconnect/  
也是可行的，原理都一樣。至於要怎麼不必開 F12 方法相同

例如上面是使用
```javascript
function ClickConnect() {
    console.log("Working");
    document.querySelector("colab-connect-button")
            .shadowRoot
            .getElementById("connect").click();
}

setInterval(ClickConnect, 60000);
```

那就是把整堆 code 前面加上 `avascript:`   
最後手動加上 `j`，整個貼到網址列就行了～

### 🚧 **Issues**
+ [ ] 無法防止「我不是機器人」的情況
> 如果有誰知道怎麼辦的，歡迎上 NTU COOL 分享～